In [1]:
import glob
from os import path
import json
import numpy as np
from scipy.stats import sem
import pandas as pd

In [2]:
abl_fp = "/data/b2p-siteident/experiments/new_baseline_rgb"

In [3]:
header = [
    "version", "freeze", "tile_size", "pretrained"
]
metrics = [
    "test_weighted_f1", 
    # "test_balanced_acc",
    "test_rwanda_weighted_f1",
    # "test_rwanda_balanced_acc",
    "test_uganda_weighted_f1",
    # "test_uganda_balanced_acc",
    "ecount"
]

entries_dict = {}
for exp_fp in sorted(glob.glob(path.join(abl_fp, "*"))):
    if not path.isfile(path.join(exp_fp, "test_stats.json")):
            print("{} not finished.".format(exp_fp))
            continue
    with open(path.join(exp_fp, "opts.json")) as f:
        opts = json.load(f)
    
    with open(path.join(exp_fp, "test_stats.json")) as f:
        stats = json.load(f)
    
    data_version = opts["dataset_name"].split("_")[0]
    freeze = opts["use_last_n_layers"]
    tile_size = opts["dataset_name"].split("_")[-2]
    pretrained = not opts["no_use_pretrained"]

    key = (data_version, freeze, tile_size, pretrained)
    if key not in entries_dict:
        entries_dict[key] = {m: [] for m in metrics}
        entries_dict[key]["ecount"] = 0
        entries_dict[key]["path"] = exp_fp
    for m in metrics:
        if m == "ecount":
            continue
        entries_dict[key][m].append(stats[m])
    entries_dict[key]["ecount"] += 1

entries = []
i = 0
for k, entry in entries_dict.items():
    avg_entry = list(k)
    for m in metrics:
        if m == "ecount":
            avg_entry.append(entry[m])
        else:
            val = np.array(entry[m])
            if "acc" in m:
                val *= 100
            avg_entry.append(round(np.mean(val), 2))
            avg_entry.append(round(sem(val), 2))
    entries.append(avg_entry)
    i += 1

metrics_header = []
for m in metrics:
    if m == "ecount":
        metrics_header.append(m)
    else:
        metrics_header.append(m + "_m")
        metrics_header.append(m + "_ste")
df = pd.DataFrame(entries, columns=header + metrics_header)

In [4]:
df[(df.ecount == 3) & (df.tile_size == "1200")].sort_values(by=["tile_size", "freeze", "version"])

,version,freeze,tile_size,pretrained,test_weighted_f1_m,test_weighted_f1_ste,test_rwanda_weighted_f1_m,test_rwanda_weighted_f1_ste,test_uganda_weighted_f1_m,test_uganda_weighted_f1_ste,ecount
0,v1,-1,1200,False,0.56,0.01,0.47,0.01,0.69,0.01,3
2,v2,-1,1200,False,0.48,0.01,0.41,0.01,0.84,0.02,3
1,v1,9,1200,False,0.56,0.01,0.49,0.01,0.66,0.02,3
3,v2,9,1200,False,0.52,0.00,0.45,0.02,0.84,0.05,3


\res{0.49}{0.01} & \res{0.66}{0.01} & \res{0.56}{0.01} & & \res{0.45}{0.02} & \res{}{0.64} & \res{0.52}{<0.01}